In [1]:
import pandas as pd
import requests
import csv
import json
from collections import OrderedDict

In [2]:
language_data = pd.read_csv("DDW-C18-0000.csv")

In [3]:
language_data.head(7)

,"C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State,District,Area Name,Total/,Age-group,Number speaking second language,NaN,NaN,Number speaking third language,NaN,NaN
1,code,code,NaN,Rural/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Urban,NaN,Persons,Males,Females,Persons,Males,Females
3,1,2,3,4,5,6,7,8,9,10,11
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957


### Filter the dataframe for useful data and modify column headers

In [4]:
language_data = language_data.loc[5:,:]

In [5]:
['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3']

['State_Code',
 'District_Code',
 'Area_Name',
 'Total/Rural/Urban',
 'Age-group',
 'Number_speaking_second_language',
 'Males2',
 'Females2',
 'Number_speaking_third_language',
 'Males3',
 'Females3']

In [6]:
language_data_total = language_data.set_axis(['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3'], axis=1, inplace=False)
language_data_total

,State_Code,District_Code,Area_Name,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957
7,00,00,INDIA,Total,10-14,34488492,18133423,16355069,7254335,3831131,3423204
8,00,00,INDIA,Total,15-19,42424599,22750908,19673691,12626717,6792766,5833951
9,00,00,INDIA,Total,20-24,41344406,22386694,18957712,12834334,7067614,5766720
...,...,...,...,...,...,...,...,...,...,...,...
1080,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,25-29,13065,7012,6053,4364,2355,2009
1081,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,30-49,38406,21301,17105,13320,7824,5496
1082,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,50-69,12899,7903,4996,4177,2842,1335
1083,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,70+,1644,1016,628,432,305,127


In [7]:
indexNames = language_data_total[language_data_total['Area_Name'] == 'INDIA' ].index
language_data = language_data_total.drop(indexNames , inplace=False)

In [8]:
data = pd.read_csv("census_data.csv")
data

,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,TRU,No_HH,...,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F
0,0,0,0,0,0,0,India,India,Total,249501663,...,5820403,1420348,491424,928924,5518450,3082613,2435837,728966109,291330383,437635726
1,0,0,0,0,0,0,India,India,Rural,168612897,...,5640822,1154975,386077,768898,3803400,1972348,1831052,485005760,200944045,284061715
2,0,0,0,0,0,0,India,India,Urban,80888766,...,179581,265373,105347,160026,1715050,1110265,604785,243960349,90386338,153574011
3,1,0,0,0,0,0,STATE,JAMMU & KASHMIR,Total,2119718,...,44570,26585,6780,19805,108457,56880,51577,8218589,3445572,4773017
4,1,0,0,0,0,0,STATE,JAMMU & KASHMIR,Rural,1553433,...,40936,21963,5757,16206,89232,45625,43607,5994979,2562471,3432508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,35,639,0,0,0,0,DISTRICT,North & Middle Andaman,Rural,25475,...,119,30,21,9,1047,654,393,65311,23633,41678
2024,35,639,0,0,0,0,DISTRICT,North & Middle Andaman,Urban,724,...,1,0,0,0,70,43,27,1707,616,1091
2025,35,640,0,0,0,0,DISTRICT,South Andaman,Total,59064,...,63,48,24,24,1311,933,378,141311,49720,91591
2026,35,640,0,0,0,0,DISTRICT,South Andaman,Rural,23767,...,62,42,20,22,777,550,227,57596,19971,37625


In [9]:
data = data[["Level","Name","TRU","TOT_P","TOT_M","TOT_F"]]

In [10]:
df = data[data['Level'] == "STATE"]

In [11]:
census_dataset = df[df['TRU'] == "Total"]
census_dataset

,Level,Name,TRU,TOT_P,TOT_M,TOT_F
3,STATE,JAMMU & KASHMIR,Total,12541302,6640662,5900640
72,STATE,HIMACHAL PRADESH,Total,6864602,3481873,3382729
111,STATE,PUNJAB,Total,27743338,14639465,13103873
174,STATE,CHANDIGARH,Total,1055450,580663,474787
180,STATE,UTTARAKHAND,Total,10086292,5137773,4948519
222,STATE,HARYANA,Total,25351462,13494734,11856728
288,STATE,NCT OF DELHI,Total,16787941,8987326,7800615
318,STATE,RAJASTHAN,Total,68548437,35550997,32997440
420,STATE,UTTAR PRADESH,Total,199812341,104480510,95331831
636,STATE,BIHAR,Total,104099452,54278157,49821295


### Calculating State Level Data

In [12]:
df1 = language_data.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
Area_Name,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,00,Total,Total,257425,142378,115047,72708,41846,30862
ANDHRA PRADESH,28,00,Total,Total,21583334,12193992,9389342,6344376,3836382,2507994
ARUNACHAL PRADESH,12,00,Total,Total,885945,478413,407532,418551,236666,181885
ASSAM,18,00,Total,Total,14460591,7979404,6481187,4352497,2636287,1716210
BIHAR,10,00,Total,Total,16761785,9443495,7318290,1682572,1028475,654097
CHANDIGARH,04,00,Total,Total,579920,322898,257022,321979,178557,143422
CHHATTISGARH,22,00,Total,Total,3384223,1820621,1563602,422569,245150,177419
DADRA & NAGAR HAVELI,26,00,Total,Total,198983,113211,85772,71516,44256,27260
DAMAN & DIU,25,00,Total,Total,116629,71260,45369,34602,21856,12746


In [13]:
group_state = list(df1.groups.keys())
group_state

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [14]:
State_dict = dict()

In [15]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z = z.iloc[0]
    Number_speaking_second_language = int(z['Number_speaking_second_language'])
    Number_speaking_third_language = int(z['Number_speaking_third_language'])
    State_Code = z['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    population = population_row['TOT_P'].tolist()[0]
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("Number_speaking_second_language = ",Number_speaking_second_language)
    #print("Number_speaking_third_language = ",Number_speaking_third_language)
    #print("population = ",population)
    exactly_one_lang = ((population - Number_speaking_second_language)*100)/population
    exactly_two_lang = ((Number_speaking_second_language - Number_speaking_third_language)*100)/population
    three_or_more_lang = (Number_speaking_third_language*100)/population
    #print("")
    #print("exactly_one_lang = ",exactly_one_lang)
    #print("exactly_two_lang = ",exactly_two_lang)
    #print("three_or_more_lang = ",three_or_more_lang)
    State_dict[State_Code] = [round(exactly_one_lang,3),round(exactly_two_lang,3),round(three_or_more_lang,3)]

In [16]:
State_dict = OrderedDict(sorted(State_dict.items())) # Order the ouput by first column i.e. key
State_dict

OrderedDict([('01', [50.697, 32.588, 16.715]),
             ('02', [81.897, 13.044, 5.059]),
             ('03', [53.015, 18.762, 28.223]),
             ('04', [45.055, 24.439, 30.506]),
             ('05', [82.573, 15.565, 1.862]),
             ('06', [77.803, 17.58, 4.617]),
             ('07', [59.392, 32.517, 8.091]),
             ('08', [89.098, 9.433, 1.469]),
             ('09', [88.552, 10.153, 1.294]),
             ('10', [83.898, 14.485, 1.616]),
             ('11', [36.29, 34.0, 29.71]),
             ('12', [35.974, 33.778, 30.248]),
             ('13', [37.846, 34.988, 27.166]),
             ('14', [51.375, 27.026, 21.599]),
             ('15', [71.773, 23.399, 4.827]),
             ('16', [65.47, 29.125, 5.405]),
             ('17', [72.283, 18.45, 9.267]),
             ('18', [53.66, 32.392, 13.948]),
             ('19', [83.538, 13.306, 3.156]),
             ('20', [69.684, 25.487, 4.829]),
             ('21', [67.063, 19.773, 13.164]),
             ('22', [86.752, 11.59

In [17]:
a_file = open("percent-india-a.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'percent-one', 'percent-two', 'percent-three'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [18]:
data1 = pd.read_csv("percent-india-a.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data")
print("")
print(data1)

Displaying states and union territories of India data

 state-code  percent-one  percent-two  percent-three
          1       50.697       32.588         16.715
          2       81.897       13.044          5.059
          3       53.015       18.762         28.223
          4       45.055       24.439         30.506
          5       82.573       15.565          1.862
          6       77.803       17.580          4.617
          7       59.392       32.517          8.091
          8       89.098        9.433          1.469
          9       88.552       10.153          1.294
         10       83.898       14.485          1.616
         11       36.290       34.000         29.710
         12       35.974       33.778         30.248
         13       37.846       34.988         27.166
         14       51.375       27.026         21.599
         15       71.773       23.399          4.827
         16       65.470       29.125          5.405
         17       72.283       18.450       

### Calculating Country Level Data

In [19]:
indexNames = language_data_total[language_data_total['Area_Name'] != 'INDIA' ].index
language_data = language_data_total.drop(indexNames , inplace=False)
language_data = language_data.iloc[0]
language_data

State_Code                                00
District_Code                             00
Area_Name                              INDIA
Total/Rural/Urban                      Total
Age-group                              Total
Number_speaking_second_language    314988770
Males2                             176696383
Females2                           138292387
Number_speaking_third_language      86009580
Males3                              50536832
Females3                            35472748
Name: 5, dtype: object

In [20]:
df = data[data['Level'] == "India"]

In [21]:
census_dataset = df[df['TRU'] == "Total"]
census_dataset

,Level,Name,TRU,TOT_P,TOT_M,TOT_F
0,India,India,Total,1210854977,623270258,587584719


In [22]:
Number_speaking_second_language = int(language_data['Number_speaking_second_language'])
Number_speaking_third_language = int(language_data['Number_speaking_third_language'])
population = census_dataset['TOT_P'].tolist()[0]

In [23]:
#print("Number_speaking_second_language = ",Number_speaking_second_language)
#print("Number_speaking_third_language = ",Number_speaking_third_language)
#print("population = ",population)

In [24]:
exactly_one_lang = ((population - Number_speaking_second_language)*100)/population
exactly_two_lang = ((Number_speaking_second_language - Number_speaking_third_language)*100)/population
three_or_more_lang = (Number_speaking_third_language*100)/population
#print("exactly_one_lang = ",exactly_one_lang)
#print("exactly_two_lang = ",exactly_two_lang)
#print("three_or_more_lang = ",three_or_more_lang)

In [25]:
Country_dict = dict()

In [26]:
Country_dict['INDIA'] = [round(exactly_one_lang,3),round(exactly_two_lang,3),round(three_or_more_lang,3)]

In [27]:
Country_dict

{'INDIA': [73.986, 18.911, 7.103]}

In [28]:
print("")

In [29]:
a_file = open("percent-india-b.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['Country', 'percent-one', 'percent-two', 'percent-three'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in Country_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [30]:
data = pd.read_csv("percent-india-b.csv")
data = data.to_string(index=False)
print("Displaying over all country data")
print("")
print(data)

Displaying over all country data

Country  percent-one  percent-two  percent-three
  INDIA       73.986       18.911          7.103
